In [1]:
%%writefile .env
POLYGON_API_KEY=l7eyBqnxp9XsobMxIBIVHx69zqRlY5rc
COHERE_API_KEY=xFDQUTLDdIZJQ4fIyJaLoKtYZTeBvIFpInEAmAbg
NEWS_API_KEY = 7bd98c5b274c406180643c142b572c68

Overwriting .env


### NEWS Search Module(not necessary)

In [2]:
%%writefile news_api_module.py
from dotenv import load_dotenv
import requests
import os
from datetime import datetime, timedelta

load_dotenv()

NEWS_API_KEY = os.getenv('NEWS_API_KEY')

def newsapi_search_news(company_name: str):
    url = "https://newsapi.org/v2/everything"
    
    # 获取当月日期范围
    current_date = datetime.now()
    from_date = current_date.replace(day=1).strftime('%Y-%m-%d')
    
    headers = {
        "X-API-Key": NEWS_API_KEY
    }
    
    params = {
        "q": f"{company_name} AND (stock OR finance OR earnings OR market)",
        "from": from_date,
        "language": "en",
        "sortBy": "relevancy",
        "pageSize": 100
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    
    # 过滤当月文章
    current_month = current_date.month
    current_year = current_date.year
    
    filtered_articles = []
    for article in data.get("articles", []):
        if article["description"]:
            pub_date = datetime.fromisoformat(article["publishedAt"].replace('Z', '+00:00'))
            if pub_date.month == current_month and pub_date.year == current_year:
                filtered_articles.append({
                    'description': article['description'],
                    'title': article['title'],
                    'datePublished': article['publishedAt']
                })
    
    return filtered_articles

Writing news_api_module.py


In [3]:
%%writefile data_module.py
from dotenv import load_dotenv
import requests
import os

load_dotenv()  # This loads the variables from .env

POLYGON_API_KEY = os.getenv('POLYGON_API_KEY')

# Monthly Stock Price History Data
def get_stock_price_history(ticker: str, start_date: str, end_date: str):
  # Polygon.io API URL for AAPL stock data
  url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/month/{start_date}/{end_date}?apiKey={POLYGON_API_KEY}'

  # Make the GET request
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the JSON data
      data = response.json()
      return [result['vw'] for result in data['results']]
  else:
      print(f'Failed to retrieve data: {response.status_code}')

Writing data_module.py


In [ ]:
%%writefile date_utils.py
import datetime
import calendar

def get_first_last_days():
    # Get the current date
    current_date = datetime.date.today()

    # Calculate 5 months ago (for start date: 5 months ago, day 1)
    start_year, start_month = current_date.year, current_date.month
    for _ in range(5):  # Go back 5 months
        if start_month == 1:
            start_month = 12
            start_year -= 1
        else:
            start_month -= 1
    
    # Start date: first day of 5 months ago
    start_date = datetime.date(start_year, start_month, 1)

    # Calculate last month (for end date: last month, last day)
    if current_date.month == 1:
        last_month = 12
        last_year = current_date.year - 1
    else:
        last_month = current_date.month - 1
        last_year = current_date.year
    
    # End date: last day of last month
    last_day_of_last_month = calendar.monthrange(last_year, last_month)[1]
    end_date = datetime.date(last_year, last_month, last_day_of_last_month)

    # Format the dates
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Generate month labels: 4 months starting from 4 months ago (skip the 5 months ago start month)
    year_month_labels = []
    year, month = start_year, start_month
    # Skip first month (5 months ago), start from 4 months ago
    month += 1
    if month > 12:
        month = 1
        year += 1
    
    for _ in range(4):  # 4 consecutive months: 4,3,2,1 months ago
        year_month_labels.append(f"{year}-{str(month).zfill(2)}")
        month += 1
        if month > 12:
            month = 1
            year += 1

    return start_date_str, end_date_str, year_month_labels

Writing calendar.py


In [6]:
%%writefile model.py

import torch
from torch import nn
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
# 只导入需要的transformers模块，避免TensorFlow冲突
from transformers import BertTokenizer, BertForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class StockLSTMRegressor(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        """
        LSTM模型用于股价预测
        Args:
            input_size: 输入特征数量（涨幅 + 情感指标 = 2）
            hidden_size: LSTM隐藏层大小
            num_layers: LSTM层数
            output_size: 输出大小（1表示预测下个月的涨幅）
            dropout: dropout率
        """
        super(StockLSTMRegressor, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                           dropout=dropout if num_layers > 1 else 0,
                           batch_first=True)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Linear layer for final prediction
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        # sequence_length = 4 (过去4个月)
        # input_size = 2 (涨幅 + 情感指标)

        # 初始化隐藏状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTM forward pass
        lstm_out, _ = self.lstm(x, (h0, c0))

        # 取最后一个时间步的输出
        last_output = lstm_out[:, -1, :]

        # Dropout
        out = self.dropout(last_output)

        # Linear layer
        out = self.linear(out)

        return out

def calculate_sentiment_score(headlines, finbert_model, tokenizer):

    if not headlines:
        return 0.0

    sentiment_scores = []
    
    try:
        device = next(finbert_model.parameters()).device
    except:
        device = torch.device("cpu")

    for headline in headlines:
        try:
            # 使用FinBERT计算情感得分
            encoded = tokenizer.encode_plus(
                headline,
                add_special_tokens=True,
                max_length=64,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            # 移动到正确的设备
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)

            with torch.no_grad():
                outputs = finbert_model(input_ids=input_ids, attention_mask=attention_mask)

                logits = outputs.logits
                probabilities = torch.softmax(logits, dim=-1).cpu().numpy()[0]

                # FinBERT输出: [negative, neutral, positive]
                # 转换为连续的情感得分: positive - negative
                weighted_score = probabilities[0] - probabilities[1]
                sentiment_scores.append(weighted_score)

        except Exception as e:
            print(f"处理标题时出错: {headline[:50]}... 错误: {e}")
            # 出错时使用中性得分
            sentiment_scores.append(0.0)

    return float(np.mean(sentiment_scores)) if sentiment_scores else 0.0

Writing model.py


In [ ]:
%%writefile main.py
import torch
import numpy as np
import os
import calendar
import requests
from date_utils import get_first_last_days
from data_module import get_stock_price_history
from model import StockLSTMRegressor, calculate_sentiment_score

# 全局变量（会被外部程序设置）
API_KEY = "l7eyBqnxp9XsobMxIBIVHx69zqRlY5rc"
base_url = "https://api.massive.com/v2/reference/news"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def calculate_monthly_returns(prices):
    """计算月度涨幅"""
    if not prices or len(prices) < 2:
        return []
    
    returns = []
    for i in range(1, len(prices)):
        monthly_return = (prices[i] - prices[i-1]) / prices[i-1]
        returns.append(monthly_return)
    return returns

def predict_stock_end_to_end(ticker_symbol: str, model_path: str = r"C:\Users\14869\Desktop\Fintech\Pro_web\model\LSTM_FINTECH.pth"):
    result = {
        'ticker': ticker_symbol,
        'success': False,
        'predicted_return': None,
        'predicted_price': None,
        'current_price': None,
        'monthly_data': {},
        'news_by_month': {},
        'error': None
    }
    
    try:
        # 获取全局变量（由 streamlit 应用设置）
        import builtins
        finbert_model = getattr(builtins, 'finbert_model', None)
        tokenizer = getattr(builtins, 'tokenizer', None)
        api_key = getattr(builtins, 'API_KEY', API_KEY)
        base_url_global = getattr(builtins, 'base_url', base_url)
        device_global = getattr(builtins, 'device', device)
        
        print("📅 第1步：获取日期范围")
        start_date, end_date, target_months = get_first_last_days()
        
        print(f"\n📰 第2步：搜索 {ticker_symbol} 的新闻数据")
        news_by_month = {}
        
        for month_label in target_months:
            year, month = month_label.split('-')
            last_day = calendar.monthrange(int(year), int(month))[1]
            
            params = {
                "ticker": ticker_symbol,
                "published_utc.gte": f"{year}-{month}-01T00:00:00Z",
                "published_utc.lte": f"{year}-{month}-{last_day:02d}T23:59:59Z",
                "limit": 30,
                "order": "descending",
                "sort": "published_utc",
            }
            
            headers = {"Authorization": f"Bearer {api_key}"}
            resp = requests.get(base_url_global, params=params, headers=headers)
            resp.raise_for_status()
            data = resp.json()
            
            news_by_month[month_label] = {
                'count': data.get("count", 0),
                'articles': data.get("results", [])
            }
        
        total_news = sum(data['count'] for data in news_by_month.values())
        print(f"获取到 {total_news} 篇新闻")
        
        print(f"\n💭 第3步：计算情感得分")
        monthly_sentiment_scores = {}
        
        for month_label, month_data in news_by_month.items():
            articles = month_data['articles']
            
            headlines = []
            for article in articles:
                title = article.get('title', '')
                description = article.get('description', '')
                headline = title if title else description
                if headline:
                    headlines.append(headline)
            
            sentiment_score = calculate_sentiment_score(headlines, finbert_model, tokenizer)
            monthly_sentiment_scores[month_label] = sentiment_score
            print(f"{month_label}: {len(headlines)} 条新闻, 情感得分: {sentiment_score:.3f}")

        print(f"\n📈 第4步：获取 {ticker_symbol} 股价数据")
        stock_prices = get_stock_price_history(ticker_symbol, start_date, end_date)
        
        if not stock_prices:
            raise ValueError(f"无法获取 {ticker_symbol} 的股价数据")
        
        print(f"获取到 {len(stock_prices)} 个月的股价数据")
        
        # 第5步：计算月度涨幅
        print(f"\n📊 第5步：计算月度涨幅")
        monthly_returns = calculate_monthly_returns(stock_prices)
        
        if len(monthly_returns) != 4:
            raise ValueError(f"期望4个月涨幅数据，实际获得 {len(monthly_returns)} 个")
        
        # 第6步：准备LSTM输入数据
        print(f"\n🔧 第6步：准备LSTM输入数据")
        sentiment_values = list(monthly_sentiment_scores.values())
        
        # 创建特征矩阵
        features = []
        for i, month in enumerate(target_months):
            month_features = [
                monthly_returns[i],  # 月度涨幅
                sentiment_values[i]  # 情感得分
            ]
            features.append(month_features)
        
        features_array = np.array(features, dtype=np.float32)
        print(f"特征矩阵形状: {features_array.shape}")
        
        # 第7步：加载LSTM模型
        print(f"\n🤖 第7步：加载LSTM模型")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"模型文件不存在: {model_path}")
        
        lstm_model = StockLSTMRegressor(
            input_size=2, hidden_size=64, num_layers=2, output_size=1, dropout=0.2
        )
        
        checkpoint = torch.load(model_path, map_location=device_global)
        if 'model_state_dict' in checkpoint:
            lstm_model.load_state_dict(checkpoint['model_state_dict'])
        else:
            lstm_model.load_state_dict(checkpoint)
        
        lstm_model = lstm_model.to(device_global)
        lstm_model.eval()
        
        # 第8步：进行预测
        print(f"\n🔮 第8步：进行股价预测")
        input_tensor = torch.from_numpy(features_array).unsqueeze(0).to(device_global)
        
        with torch.no_grad():
            prediction = lstm_model(input_tensor)
            predicted_return = prediction.item()
        
        # 计算预测价格
        current_price = stock_prices[-1]
        predicted_price = current_price * (1 + predicted_return)
        
        # 保存结果
        result.update({
            'success': True,
            'predicted_return': predicted_return,
            'predicted_price': predicted_price,
            'current_price': current_price,
            'monthly_data': {
                'months': target_months,
                'returns': monthly_returns,
                'sentiment_scores': sentiment_values,
                'prices': stock_prices
            },
            'news_by_month': news_by_month
        })
        
        # 第9步：显示预测结果
        print(f"\n🎯 预测结果")
        print("=" * 60)
        print(f"📊 股票代码: {ticker_symbol}")
        print(f"💰 当前价格: ${current_price:.2f}")
        print(f"🎯 预测价格: ${predicted_price:.2f}")
        print(f"📈 预测涨幅: {predicted_return:.4f} ({predicted_return*100:.2f}%)")
        
        if predicted_return > 0:
            print("✅ 模型预测股价将上涨")
        else:
            print("⚠️ 模型预测股价将下跌")
        
        print(f"\n📋 输入特征详情:")
        for i, month in enumerate(target_months):
            print(f"   {month}: 涨幅={monthly_returns[i]:+.4f}, 情感得分={sentiment_values[i]:+.4f}")
            
    except Exception as e:
        error_msg = f"预测过程中出错: {str(e)}"
        print(f"❌ {error_msg}")
        result['error'] = error_msg
        import traceback
        traceback.print_exc()
    
    return result

Writing main.py


In [8]:
%%writefile utils.py
import numpy as np
import os
import torch
import calendar
import requests

# 全局变量（会被 streamlit 应用设置）
API_KEY = "l7eyBqnxp9XsobMxIBIVHx69zqRlY5rc"
base_url = "https://api.massive.com/v2/reference/news"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def calculate_monthly_returns(prices):
    """计算月度涨幅"""
    if not prices or len(prices) < 2:
        return []
    
    returns = []
    for i in range(1, len(prices)):
        monthly_return = (prices[i] - prices[i-1]) / prices[i-1]
        returns.append(monthly_return)
    return returns

Writing utils.py


In [ ]:
# 安装必要的依赖包
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 安装成功")
    except subprocess.CalledProcessError as e:
        print(f"❌ {package} 安装失败: {e}")

# 安装 python-dotenv
install_package("python-dotenv")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'C:\Users\14869\Desktop\Fintech\Pro_web\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:

!streamlit run streamlit_main.py




^C


2025-11-19 17:47:31.536 Uncaught app execution
Traceback (most recent call last):
  File "c:\Users\14869\Desktop\Fintech\Pro_web\venv\lib\site-packages\streamlit\runtime\scriptrunner\exec_code.py", line 128, in exec_func_with_error_handling
    result = func()
  File "c:\Users\14869\Desktop\Fintech\Pro_web\venv\lib\site-packages\streamlit\runtime\scriptrunner\script_runner.py", line 669, in code_to_exec
    exec(code, module.__dict__)  # noqa: S102
  File "c:\Users\14869\Desktop\Fintech\Pro_web\streamlit_main.py", line 44, in <module>
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NameError: name 'torch' is not defined



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.31.53:8501

